In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# Create cluster in Cassandra: This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
# Create a Keyspace  
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
## Confirmation Query to text:  Find artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_app_history_byidanditeminsession "
query = query + "(  sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

In [9]:
# Find file, create loop to insert rows, add insert statements into 'query' variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history_byidanditeminsession ( sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## Query Description: In this query, I used `sessionid` as the partition key and `itemInSession` as my clustering key. Each partition \
## is uniquely identified by `sessionid` while `itemInSession` was used to uniquely identify the rows within a partition to sort the data by \
## the value of int.
query = "select artist, song, length from music_app_history_byidanditeminsession WHERE sessionid = 338 AND iteminsession = 4 "
rows = session.execute(query)

for row in rows:
    print (row.artist, row.song, row.length )

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
## Confirmation Query 1: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query1 = "CREATE TABLE IF NOT EXISTS music_app_history_byuseridandsessionid "
query1 = query1 + "( userId int, sessionId int,  itemInSession int, artist text, song text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)  

                    

In [12]:
# Find file, create loop to insert rows, add insert statements into 'query' variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO music_app_history_byuseridandsessionid ( userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## Query Description: In this query, I used `userId` and 'sessionId' as the composite partition key and `itemInSession` as my clustering key. \
## Each partition is uniquely identified by 'userId' and `sessionid` while `itemInSession` was used to uniquely identify the rows within a partition\
## to sort the data by the value of int.
query1 = "select artist, song, userid, firstname, lastname from music_app_history_byuseridandsessionid WHERE userid = 10 AND sessionid = 182 "
rows = session.execute(query1)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname )


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## Confirmation Query 2: Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query2 = "CREATE TABLE IF NOT EXISTS music_app_history_byusernameandsong "
query2 = query2 + "( song text, userid int, firstName text, lastName text, PRIMARY KEY (song, userid))"
try:
    session.execute(query2)
except Exception as e:
    print(e)  

                    

In [15]:
# Find file, create loop to insert rows, add insert statements into 'query' variable
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO music_app_history_byusernameandsong ( song, userid, firstName, lastName)"
        query2 = query2 + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query2, ( line[9], int(line[10]), line[1], line[4]))

In [16]:
## Query Description: In this query, I used `song` as the partition key and `userid` as my clustering key. \
## Each partition is uniquely identified by `song` while `itemInSession` was used to uniquely identify the rows within a partition\
## to sort the data by the value of int.
query2 = "select firstname, lastname from music_app_history_byusernameandsong WHERE song = 'All Hands Against His Own' "
rows = session.execute(query2)

for row in rows:
    print (row.firstname, row.lastname )

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [17]:
## Drop the table before closing out the sessions
query = "drop table music_app_history_byidanditeminsession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query1 = "drop table music_app_history_byuseridandsessionid"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query1 = "drop table music_app_history_byusernameandsong"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [18]:
session.shutdown()
cluster.shutdown()